<a href="https://colab.research.google.com/github/namwootree/Portfolio/blob/main/Alphaco_(Deep_Learning_Boot_Camp)/Long-Term%20Program/Text_Summarization/%5BEDA_%EA%B8%B0%EB%B2%95%5DData_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting

In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 11.8 MB/s 
     |████████████████████████████████| 596 kB 45.4 MB/s 
     |████████████████████████████████| 86 kB 5.0 MB/s 
     |████████████████████████████████| 6.6 MB 49.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 12.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

import transformers
from transformers import pipeline
import random

import nltk
from nltk.tokenize import sent_tokenize

from tqdm.notebook import tqdm

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
unmasker = pipeline('fill-mask', model='klue/roberta-base')

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/734k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/장기 프로젝트/문서 요약/train_df.csv')

In [ ]:
train = train[['context', 'summary']]
train['context'] = train['context'].str.replace('.', '. ').replace('?', '? ').replace('없습니다.', '없습니다')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  


In [ ]:
train.shape

(2994, 2)

In [ ]:
train.head()

,context,summary
0,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제207회 완주군의회 임시회 ...,제207회 완주군의회 임시회 제1차 본회의 개의 선포.
1,의사팀장 수고하셨습니다. 먼저 의사일정 제1항 제207회 완주군의회 임시회 회기 ...,제207회 완주군의회 임시회 회기는 8월 26일부터 9월 4일까지 10일간으로 가결됨.
2,다음은 의사일정 제2항 제207회 완주군의회 임시회 회의록 서명의원 선출의 건을 상...,제207회 완주군의회 임시회 회의록 서명의원으로 최등원 의원과 박웅배 의원이 선출됨.
3,다음은 의사일정 제3항 본회의 휴회의 건을 상정합니다. 상임의원회 의정활동을 위하...,8월 27일부터 9월 3일까지 8일간 휴회가 가결됨. 제2차 본회의는 9월 4일 오...
4,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제251회 완주군의회 제1차 ...,제251회 완주군의회 제1차 정례회 제1차 본회의 개의 선포.


# Random Insertion

In [ ]:
def random_insertion(table, unmasker):
  
  dict_random_insertion = {
      'context':[],
      'summary':list(table['summary'])
  }

  for input_text in tqdm(list(table['context'])):

    new_text = ''
    
    sentence_list = sent_tokenize(input_text)

    for sentence in sentence_list:

      orig_text_list = sentence.split()
      len_input = len(orig_text_list)
      
      if len_input == 1:
        new_text += ' ' + sentence
      
      elif len_input <= 3:

        new_text_list = orig_text_list[:1] + ['[MASK]'] + orig_text_list[1:]
        new_mask_sent = ' '.join(new_text_list)

        try:
          augmented_text_list = unmasker(new_mask_sent)
          augmented_text = augmented_text_list[0]['sequence']

          new_augmented_text = ' ' + augmented_text
          new_text += new_augmented_text
          new_text = new_text.lstrip(' ')
        except:
          print(f'오류 : {new_mask_sent}')
          new_text += ' ' + sentence
          pass

      else:
        rand_idx = random.randint(1, len_input-2)

        new_text_list = orig_text_list[:rand_idx] + ['[MASK]'] + orig_text_list[rand_idx:]
        new_mask_sent = ' '.join(new_text_list)

        try:
          augmented_text_list = unmasker(new_mask_sent)
          augmented_text = augmented_text_list[0]['sequence']

          new_augmented_text = ' ' + augmented_text
          new_text += new_augmented_text
          new_text = new_text.lstrip(' ')
        except:
          print(f'오류 : {new_mask_sent}')
          new_text += ' ' + sentence
          pass

    dict_random_insertion['context'].append(new_text+'.')

  df_random_insertion = pd.DataFrame(dict_random_insertion)
  
  return df_random_insertion

In [ ]:
df_random_insertion = random_insertion(train, unmasker)

# Random Replacement

In [ ]:
def random_replacement(table, unmasker):
  
  dict_random_replacement = {
      'context':[],
      'summary':list(table['summary'])
  }

  for input_text in tqdm(list(table['context'])):


    new_text = ''
    
    sentence_list = sent_tokenize(input_text)

    for sentence in sentence_list:

      orig_text_list = sentence.split()
      len_input = len(orig_text_list)
      

      if len_input <= 2:
        new_text += ' ' + sentence

      elif len_input <= 3:
        orig_word = orig_text_list[1]
        new_text_list = orig_text_list.copy()
        new_text_list[1] = '[MASK]'
        new_mask_sent = ' '.join(new_text_list)

        try:
          augmented_text_list = unmasker(new_mask_sent)
          for res in augmented_text_list:
            if res['token_str'] != orig_word:
              augmented_text = res['sequence']
              break

              new_augmented_text = ' ' + augmented_text
              new_text += new_augmented_text
              new_text = new_text.lstrip(' ')

        except:
          print(f'오류 : {new_mask_sent}')
          new_text += ' ' + sentence
          pass

      else:
        rand_idx = random.randint(1, len_input-2)

        orig_word = orig_text_list[rand_idx]
        new_text_list = orig_text_list.copy()
        new_text_list[rand_idx] = '[MASK]'
        new_mask_sent = ' '.join(new_text_list)

        try:
          augmented_text_list = unmasker(new_mask_sent)
          for res in augmented_text_list:
            if res['token_str'] != orig_word:
              augmented_text = res['sequence']
              break
        except:
          print(f'오류 : {new_mask_sent}')
          new_text += ' ' + sentence
          pass

        new_augmented_text = ' ' + augmented_text
        new_text += new_augmented_text
        new_text = new_text.lstrip(' ')

    dict_random_replacement['context'].append(new_text+'.')

  df_random_replacement = pd.DataFrame(dict_random_replacement)

  return df_random_replacement

In [ ]:
df_random_replacement = random_replacement(train, unmasker)

# Random Delection

In [120]:
def random_delection(table):
  
  dict_random_delection = {
      'context':[],
      'summary':list(table['summary'])
  }

  for input_text in tqdm(list(table['context'])):

    new_text = ''
    
    sentence_list = sent_tokenize(input_text)

    for sentence in sentence_list:

      orig_text_list = sentence.split()
      len_input = len(orig_text_list)
      

      if len_input <= 3:
        new_text += ' ' + sentence

      else:
        rand_idx = random.randint(1, len_input-2)

        orig_word = orig_text_list[rand_idx]
        new_text_list = orig_text_list.copy()
        new_text_list[rand_idx] = ' '
        augmented_text = ' '.join(new_text_list)
        
        new_text_list_2 = augmented_text.split()
        augmented_text = ' '.join(new_text_list_2)

        new_augmented_text = ' ' + augmented_text
        new_text += new_augmented_text
        new_text = new_text.lstrip(' ')

    dict_random_delection['context'].append(new_text+'.')

  df_random_delection = pd.DataFrame(dict_random_delection)

  return df_random_delection

In [121]:
df_random_delection = random_delection(train)

  0%|          | 0/2994 [00:00<?, ?it/s]

In [122]:
df_random_delection.head()

,context,summary
0,의석을 정돈하여 바랍니다. 성원이 되었으므로 제207회 완주군의회 제1차 본회의 개...,제207회 완주군의회 임시회 제1차 본회의 개의 선포.
1,의사팀장 수고하셨습니다. 먼저 의사일정 제1항 제207회 완주군의회 임시회 회기 결...,제207회 완주군의회 임시회 회기는 8월 26일부터 9월 4일까지 10일간으로 가결됨.
2,다음은 의사일정 제2항 제207회 완주군의회 임시회 서명의원 선출의 건을 상정합니다...,제207회 완주군의회 임시회 회의록 서명의원으로 최등원 의원과 박웅배 의원이 선출됨.
3,다음은 의사일정 제3항 본회의 건을 상정합니다. 상임의원회 의정활동을 위하여 8월 ...,8월 27일부터 9월 3일까지 8일간 휴회가 가결됨. 제2차 본회의는 9월 4일 오...
4,의석을 정돈하여 바랍니다. 성원이 되었으므로 제251회 제1차 정례회 제1차 본회의...,제251회 완주군의회 제1차 정례회 제1차 본회의 개의 선포.


In [123]:
train['context'][0]

'의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제207회 완주군의회 임시회 제1차 본회의 개의를 선포합니다. 먼저 의사팀장으로부터 의회 관련 사항에 대한 보고가 있겠습니다. 의사팀장은 보고하여 주시기 바랍니다.의사팀장 이은미 입니다. 의회관련 사항을 보고 드리겠습니다. 먼저 제207회 완주군의회 임시회 소집경위를 보고 드리겠습니다. 지방자치법 제45조 제2항의 규정에 의거 서남용 의원님 외 4인의 의원님으로부터 임시회 집회 요구가 있어 지난 8월6일 집회 공고를 하고 오늘 집회를 갖게 되었습니다. 다음은 의안접수 및 회부사항입니다. 먼저 의원발의 안건입니다. 완주군의회 의원 의정활동비등 지급에 관한 조례 일부개정 조례안, 완주군 향토문화유산 보호 및 관리 조례안, 완주군 싸움소 육성 지원에 관한 조례안 등 총 3건으로, 해당 상임의원회로 회부 하였습니다. 다음은 완주군수로부터 제출된 안건입니다. 삼례문화예술촌 설치 및 운영조례 일부개정 조례안 등 총 40건으로 해당 상임의원회로 회부하였습니다. 자세한 내용은 배부해 드린 유인물을 참조하여 주시기 바랍니다. 이상 보고를 마치겠습니다'

In [124]:
df_random_delection['context'][0]

'의석을 정돈하여 바랍니다. 성원이 되었으므로 제207회 완주군의회 제1차 본회의 개의를 선포합니다. 먼저 의사팀장으로부터 의회 관련 대한 보고가 있겠습니다. 의사팀장은 주시기 바랍니다.의사팀장 이은미 입니다. 의회관련 사항을 드리겠습니다. 먼저 제207회 완주군의회 소집경위를 보고 드리겠습니다. 지방자치법 제45조 제2항의 의거 서남용 의원님 외 4인의 의원님으로부터 임시회 집회 요구가 있어 지난 8월6일 집회 공고를 하고 오늘 집회를 갖게 되었습니다. 다음은 및 회부사항입니다. 먼저 의원발의 안건입니다. 완주군의회 의원 의정활동비등 지급에 관한 조례 일부개정 조례안, 완주군 향토문화유산 및 관리 조례안, 완주군 싸움소 육성 지원에 관한 조례안 등 총 3건으로, 해당 상임의원회로 회부 하였습니다. 다음은 제출된 안건입니다. 삼례문화예술촌 및 운영조례 일부개정 조례안 등 총 40건으로 해당 상임의원회로 회부하였습니다. 자세한 내용은 드린 유인물을 참조하여 주시기 바랍니다. 이상 보고를 마치겠습니다.'

# Random Swap

In [75]:
def random_swap(table):
  
  dict_random_swap = {
      'context':[],
      'summary':list(table['summary'])
  }

  for input_text in tqdm(list(table['context'])):
    
    sentence_list = sent_tokenize(input_text)

    len_input = len(sentence_list)

    if len_input <= 5:

      rand_idx_1 = random.randint(0, len_input-1)
      rand_idx_2 = random.randint(0, len_input-1)

      sentence_list[rand_idx_1], sentence_list[rand_idx_2] = sentence_list[rand_idx_2], sentence_list[rand_idx_1]

      augmented_text = ' '.join(sentence_list)

      dict_random_swap['context'].append(augmented_text)

    else:

      rand_idx_1 = random.randint(2, len_input-3)
      rand_idx_2 = random.randint(2, len_input-3)

      sentence_list[rand_idx_1], sentence_list[rand_idx_2] = sentence_list[rand_idx_2], sentence_list[rand_idx_1]

      augmented_text = ' '.join(sentence_list)

      dict_random_swap['context'].append(augmented_text)

  df_random_swap = pd.DataFrame(dict_random_swap)

  return df_random_swap

In [76]:
df_random_swap = random_swap(train)

  0%|          | 0/2994 [00:00<?, ?it/s]

In [82]:
train['context'][2]

'다음은 의사일정 제2항 제207회 완주군의회 임시회 회의록 서명의원 선출의 건을 상정합니다.  회의록 서명의원으로는 최등원 의원님과 박웅배 의원님을 선출하고자 하는데 의원여러분 이의 있습니까?(『없습니다』하는 의원 있음)이의가 없으므로 가결되었음을 선포합니다.  그럼 두 분 의원님께서는 본 회기동안 수고하여 주시기 바랍니다'

In [81]:
df_random_swap['context'][2]

'(『없습니다』하는 의원 있음)이의가 없으므로 가결되었음을 선포합니다. 회의록 서명의원으로는 최등원 의원님과 박웅배 의원님을 선출하고자 하는데 의원여러분 이의 있습니까? 다음은 의사일정 제2항 제207회 완주군의회 임시회 회의록 서명의원 선출의 건을 상정합니다. 그럼 두 분 의원님께서는 본 회기동안 수고하여 주시기 바랍니다'

# Augmentation

In [92]:
train = pd.read_csv('/content/drive/MyDrive/장기 프로젝트/문서 요약/train_df.csv')
SR = pd.read_csv('/content/drive/MyDrive/장기 프로젝트/문서 요약/df_random_replacement.csv')
RI = pd.read_csv('/content/drive/MyDrive/장기 프로젝트/문서 요약/df_random_insertion.csv')

In [93]:
train.shape, SR.shape, RI.shape

((2994, 8), (2994, 6), (2994, 6))

In [94]:
display(train.head(), SR.head(), RI.head())

,Unnamed: 0,id,title,region,agenda,context,summary,total
0,1000,2000,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,AGENDA_1,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제207회 완주군의회 임시회 제...,제207회 완주군의회 임시회 제1차 본회의 개의 선포.,제207회 완주군의회(임시회) 제 1 차 본회의회의록 완주 의석을 정돈하여 주시기 ...
1,1001,2000,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,AGENDA_2,의사팀장 수고하셨습니다. 먼저 의사일정 제1항 제207회 완주군의회 임시회 회기 결...,제207회 완주군의회 임시회 회기는 8월 26일부터 9월 4일까지 10일간으로 가결됨.,제207회 완주군의회(임시회) 제 1 차 본회의회의록 완주 의사팀장 수고하셨습니다....
2,1002,2000,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,AGENDA_3,다음은 의사일정 제2항 제207회 완주군의회 임시회 회의록 서명의원 선출의 건을 상...,제207회 완주군의회 임시회 회의록 서명의원으로 최등원 의원과 박웅배 의원이 선출됨.,제207회 완주군의회(임시회) 제 1 차 본회의회의록 완주 다음은 의사일정 제2항 ...
3,1003,2000,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,AGENDA_4,다음은 의사일정 제3항 본회의 휴회의 건을 상정합니다. 상임의원회 의정활동을 위하여...,8월 27일부터 9월 3일까지 8일간 휴회가 가결됨. 제2차 본회의는 9월 4일 오...,제207회 완주군의회(임시회) 제 1 차 본회의회의록 완주 다음은 의사일정 제3항 ...
4,1004,2001,제251회 완주군의회(제1차 정례회) 제1차 본 회 의 회 의 록,완주,AGENDA_1,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제251회 완주군의회 제1차 정...,제251회 완주군의회 제1차 정례회 제1차 본회의 개의 선포.,제251회 완주군의회(제1차 정례회) 제1차 본 회 의 회 의 록 완주 의석을 정돈...


,Unnamed: 0,id,title,region,context,summary
0,0,2000,제207회 완주군의회(임시회) 제 1 차 본회의회의록,AGENDA_1,"의석을 정돈하여 주기 바랍니다. 성원이 되었으므로, 완주군의회 임시회 제1차 본회의...",제207회 완주군의회 임시회 제1차 본회의 개의 선포.
1,1,2000,제207회 완주군의회(임시회) 제 1 차 본회의회의록,AGENDA_2,먼저 의사일정 제1항 제207회 국회 임시회 회기 결정의 건을 상정합니다. 제207...,제207회 완주군의회 임시회 회기는 8월 26일부터 9월 4일까지 10일간으로 가결됨.
2,2,2000,제207회 완주군의회(임시회) 제 1 차 본회의회의록,AGENDA_3,다음은 의사일정 제2항 제207회 완주군의회 임시회 본회의 서명의원 선출의 건을 상...,제207회 완주군의회 임시회 회의록 서명의원으로 최등원 의원과 박웅배 의원이 선출됨.
3,3,2000,제207회 완주군의회(임시회) 제 1 차 본회의회의록,AGENDA_4,다음은 의사일정 제3항 본회의 의 건을 상정합니다. 상임의원회 의정활동을 위하여 8...,8월 27일부터 9월 3일까지 8일간 휴회가 가결됨. 제2차 본회의는 9월 4일 오...
4,4,2001,제251회 완주군의회(제1차 정례회) 제1차 본 회 의 회 의 록,AGENDA_1,의석을 늘려 주시기 바랍니다. 성원이 되었으므로 제251회 완주군의회 제1차 정례회...,제251회 완주군의회 제1차 정례회 제1차 본회의 개의 선포.


,Unnamed: 0,id,title,region,context,summary
0,0,2000,제207회 완주군의회(임시회) 제 1 차 본회의회의록,AGENDA_1,의석을 잘 정돈하여 주시기 바랍니다. 성원이 되었으므로 제207회 전북 완주군의회 ...,제207회 완주군의회 임시회 제1차 본회의 개의 선포.
1,1,2000,제207회 완주군의회(임시회) 제 1 차 본회의회의록,AGENDA_2,의사팀장 : 수고하셨습니다. 먼저 의사일정 제1항 제207회 완주군의회 임시회 및 ...,제207회 완주군의회 임시회 회기는 8월 26일부터 9월 4일까지 10일간으로 가결됨.
2,2,2000,제207회 완주군의회(임시회) 제 1 차 본회의회의록,AGENDA_3,다음은 의사일정 제2항 제207회 완주군의회 임시회 본회의 회의록 서명의원 선출의 ...,제207회 완주군의회 임시회 회의록 서명의원으로 최등원 의원과 박웅배 의원이 선출됨.
3,3,2000,제207회 완주군의회(임시회) 제 1 차 본회의회의록,AGENDA_4,다음은 의사일정 제3항 중 본회의 휴회의 건을 상정합니다. 상임의원회 의정활동을 위...,8월 27일부터 9월 3일까지 8일간 휴회가 가결됨. 제2차 본회의는 9월 4일 오...
4,4,2001,제251회 완주군의회(제1차 정례회) 제1차 본 회 의 회 의 록,AGENDA_1,의석을 잘 정돈하여 주시기 바랍니다. 성원이 다 되었으므로 제251회 완주군의회 제...,제251회 완주군의회 제1차 정례회 제1차 본회의 개의 선포.


In [95]:
train = train[['context', 'summary']]
SR = train[['context', 'summary']]
RI = train[['context', 'summary']]

In [96]:
display(train.head(), SR.head(), RI.head())

,context,summary
0,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제207회 완주군의회 임시회 제...,제207회 완주군의회 임시회 제1차 본회의 개의 선포.
1,의사팀장 수고하셨습니다. 먼저 의사일정 제1항 제207회 완주군의회 임시회 회기 결...,제207회 완주군의회 임시회 회기는 8월 26일부터 9월 4일까지 10일간으로 가결됨.
2,다음은 의사일정 제2항 제207회 완주군의회 임시회 회의록 서명의원 선출의 건을 상...,제207회 완주군의회 임시회 회의록 서명의원으로 최등원 의원과 박웅배 의원이 선출됨.
3,다음은 의사일정 제3항 본회의 휴회의 건을 상정합니다. 상임의원회 의정활동을 위하여...,8월 27일부터 9월 3일까지 8일간 휴회가 가결됨. 제2차 본회의는 9월 4일 오...
4,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제251회 완주군의회 제1차 정...,제251회 완주군의회 제1차 정례회 제1차 본회의 개의 선포.


,context,summary
0,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제207회 완주군의회 임시회 제...,제207회 완주군의회 임시회 제1차 본회의 개의 선포.
1,의사팀장 수고하셨습니다. 먼저 의사일정 제1항 제207회 완주군의회 임시회 회기 결...,제207회 완주군의회 임시회 회기는 8월 26일부터 9월 4일까지 10일간으로 가결됨.
2,다음은 의사일정 제2항 제207회 완주군의회 임시회 회의록 서명의원 선출의 건을 상...,제207회 완주군의회 임시회 회의록 서명의원으로 최등원 의원과 박웅배 의원이 선출됨.
3,다음은 의사일정 제3항 본회의 휴회의 건을 상정합니다. 상임의원회 의정활동을 위하여...,8월 27일부터 9월 3일까지 8일간 휴회가 가결됨. 제2차 본회의는 9월 4일 오...
4,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제251회 완주군의회 제1차 정...,제251회 완주군의회 제1차 정례회 제1차 본회의 개의 선포.


,context,summary
0,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제207회 완주군의회 임시회 제...,제207회 완주군의회 임시회 제1차 본회의 개의 선포.
1,의사팀장 수고하셨습니다. 먼저 의사일정 제1항 제207회 완주군의회 임시회 회기 결...,제207회 완주군의회 임시회 회기는 8월 26일부터 9월 4일까지 10일간으로 가결됨.
2,다음은 의사일정 제2항 제207회 완주군의회 임시회 회의록 서명의원 선출의 건을 상...,제207회 완주군의회 임시회 회의록 서명의원으로 최등원 의원과 박웅배 의원이 선출됨.
3,다음은 의사일정 제3항 본회의 휴회의 건을 상정합니다. 상임의원회 의정활동을 위하여...,8월 27일부터 9월 3일까지 8일간 휴회가 가결됨. 제2차 본회의는 9월 4일 오...
4,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제251회 완주군의회 제1차 정...,제251회 완주군의회 제1차 정례회 제1차 본회의 개의 선포.


In [97]:
df = pd.concat([train, SR, RI], axis=0)
df.shape

(8982, 2)

In [99]:
df.to_csv('train_SR_RI.csv', index=False)

In [107]:
new_df = pd.DataFrame(columns=['context', 'summary'])

for i in tqdm(range(23)):
  
  deletion_df = random_delection(df)
  swap_df = random_swap(df)

  # deletion_df.to_csv(f'deletion_{i}.csv')
  # swap_df.to_csv(f'swap_{i}.csv')

  new_df = pd.concat([new_df, deletion_df, swap_df], axis=0)

final_df = pd.concat([train, new_df], axis=0)

print('데이터 증식을 완료했습니다.')

  0%|          | 0/23 [00:00<?, ?it/s]

  0%|          | 0/8982 [00:00<?, ?it/s]

  0%|          | 0/8982 [00:00<?, ?it/s]

  0%|          | 0/8982 [00:00<?, ?it/s]

  0%|          | 0/8982 [00:00<?, ?it/s]

...

  0%|          | 0/8982 [00:00<?, ?it/s]

  0%|          | 0/8982 [00:00<?, ?it/s]

  0%|          | 0/8982 [00:00<?, ?it/s]

  0%|          | 0/8982 [00:00<?, ?it/s]

데이터 증식을 완료했습니다.


In [108]:
final_df.shape

(416166, 2)

In [109]:
final_df['context'].nunique()

247756

In [114]:
final_df.drop_duplicates('context', keep='first', inplace=True)
final_df.shape

(247756, 2)

In [116]:
final_df.to_csv('/content/drive/MyDrive/장기 프로젝트/문서 요약/augmented_data_set_247756.csv')